In [2]:
import random
from sklearn.model_selection import GridSearchCV
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import precision_score, roc_auc_score, make_scorer, f1_score, recall_score, roc_auc_score
from sklearn.feature_selection import SelectKBest
from sklearn.preprocessing import StandardScaler
import os

os.environ['PYTHONHASHSEED']=str(271828)
random.seed(271828)
np.random.seed(271828)
#import tensorflow as tf
from imblearn.over_sampling import SMOTE

from sklearn.pipeline import Pipeline
from tensorflow.python.framework import ops
import tensorflow as tf
tf.random.set_seed(271828)
ops.reset_default_graph()

from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input, BatchNormalization
#from tensorflow.keras.layers import Conv2D, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC
from tensorflow.keras.callbacks import EarlyStopping

import keras.backend as K
#session_conf=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1) 
#sesCriticalSection=tf.Session(graph=tf.get_default_graph(), config=session_conf)
#K.set_session(sess)

Using TensorFlow backend.


In [3]:

def PCL_calculator(df):

    symptomatic_cutoff = 2

    intrusion = ['q6.1_INTRU', 'q6.2_DREAM', 'q6.3_FLASH', 'q6.4_UPSET', 'q6.5_PHYS']
    avoidance = ['q6.6_AVTHT', 'q6.7_AVSIT', 'q6.8_AMNES', 'q6.9_DISINT', 'q6.10_DTACH',
                 'q6.11_NUMB', 'q6.12_FUTRE']
    tred = ['q6.1_INTRU', 'q6.2_DREAM', 'q6.3_FLASH']
    only_avoidance = ['q6.6_AVTHT', 'q6.7_AVSIT', 'q6.8_AMNES']

    hypertension = ['q6.13_SLEEP', 'q6.14_ANGER', 'q6.15_CONC', 'q6.16_HYPER', 'q6.17_STRTL']

    depression = ['q6.9_DISINT', 'q6.10_DTACH', 'q6.11_NUMB', 'q6.12_FUTRE']

    df[intrusion + avoidance + hypertension].fillna(df[intrusion + avoidance + hypertension].mean(axis=1))
    intrusion_cuoff = 1
    avoidance_cuoff = 3
    hypertension_cuoff = 2
    only_avoidance_cutoff = 1
    depression_cutoff = 2
    tred_cutoff = 1

    df['sum'] = (df[intrusion + avoidance + hypertension]).sum(axis=1)

    df['intrusion'] = (df[intrusion] > symptomatic_cutoff).sum(axis=1)
    df['intrusion_cutoff'] = (df['intrusion'] >= intrusion_cuoff).astype(int)

    df['avoidance'] = (df[avoidance] > symptomatic_cutoff).sum(axis=1)
    df['avoidance_cutoff'] = (df['avoidance'] >= avoidance_cuoff).astype(int)

    df['depression'] = (df[depression] > symptomatic_cutoff).sum(axis=1)
    df['depression_cutoff'] = df['depression'] >= depression_cutoff

    df['hypertention'] = (df[hypertension] > symptomatic_cutoff).sum(axis=1)
    df['hypertention_cutoff'] = (df['hypertention'] >= hypertension_cuoff).astype(int)

    df['tred'] = (df[tred] > symptomatic_cutoff).sum(axis=1)
    df['tred_cutoff'] = df['tred'] >= tred_cutoff

    df['only_avoidance'] = (df[only_avoidance] > symptomatic_cutoff).sum(axis=1)
    df['only_avoidance_cutoff'] = df['only_avoidance'] >= only_avoidance_cutoff

    df['regression_cutoff_33'] = df['sum'] >= 33
    df['regression_cutoff_50'] = df['sum'] >= 50
    df['diagnosis'] = ((df['hypertention_cutoff']) & (df['avoidance_cutoff']) & (df['intrusion_cutoff']) & (df['sum'] >= 50))

    return df



In [4]:
path = "C:\‏‏PycharmProjects\PTSD\Data\PTSD.xlsx"
df = pd.read_excel(path)
#combine with specifics of answers
df_pcl2 = pd.read_excel("C:\‏‏PycharmProjects\PTSD\Data\questionnaire6PCL2.xlsx")
df_pcl1 = pd.read_excel("C:\‏‏PycharmProjects\PTSD\Data\questionnaire6PCL1.xlsx")
df = df.merge(df_pcl1, on="ID", how='outer')
df = df.merge(df_pcl2, suffixes=('_pcl1', '_pcl2'), on="ID", how='outer')

df_pcl3 = pd.read_excel("C:\‏‏PycharmProjects\PTSD\Data\questionnaire6PCL3.xlsx")
df_pcl3 = PCL_calculator(df_pcl3)
df = df.merge(df_pcl3.drop(['PCL3_Strict', 'pcl3', 'PCL3_Broad'], axis=1), on="ID", how='outer')
# rmoving missing Y's
df = df[~df['PCL_Strict3'].isna()]

## features 

In [5]:

features = ["age", "highschool_diploma", "dyslexia", "ADHD", "T1Acc1t", "T1Acc1n", "T1bias", "phq1", "lot1",
                "trait1",
                "state1", "PCL1", "PCL_Broad1", "PCL_Strict1", "phq2", "lot2", "trait2", "state2", "PCL2", "PCL_Broad2",
                "PCL_Strict2", "cd_risc1", "active_coping1", "planning1", "positive_reframing1", "acceptance1",
                "humor1",
                "religion1", "emotional_support1", "instrumental_support1", "self_distraction1", "denial1",
                "venting1", "substance_use1", "behavioral_disengagement1", "self_blame1", "active_coping2", "planning2",
                "positive_reframing2", "acceptance2", "humor2", "religion2", "emotional_support2",
                "instrumental_support2",
                "self_distraction2", "denial2", "venting2", "substance_use2", "behavioral_disengagement2",
                "self_blame2",
                "trauma_history8_1", "HML_5HTT", "HL_MAOA", "HML_NPY", "COMT_Hap1_recode",
                "COMT_Hap2_recode", "COMT_Hap1_LvsMH", "HML_FKBP5", 'q6.1_INTRU_pcl1', 
                'q6.2_DREAM_pcl1', 'q6.3_FLASH_pcl1', 'q6.4_UPSET_pcl1',
                  'q6.5_PHYS_pcl1', 'q6.6_AVTHT_pcl1', 'q6.7_AVSIT_pcl1', 'q6.8_AMNES_pcl1', 'q6.9_DISINT_pcl1',
                  'q6.10_DTACH_pcl1', 'q6.11_NUMB_pcl1', 'q6.12_FUTRE_pcl1', 'q6.13_SLEEP_pcl1',
                  'q6.14_ANGER_pcl1', 'q6.15_CONC_pcl1', 'q6.16_HYPER_pcl1', 'q6.17_STRTL_pcl1',
                  'q6.1_INTRU_pcl2', 'q6.2_DREAM_pcl2', 'q6.3_FLASH_pcl2', 'q6.4_UPSET_pcl2',
                  'q6.5_PHYS_pcl2', 'q6.6_AVTHT_pcl2', 'q6.7_AVSIT_pcl2', 'q6.8_AMNES_pcl2', 'q6.9_DISINT_pcl2',
                  'q6.10_DTACH_pcl2', 'q6.11_NUMB_pcl2', 'q6.12_FUTRE_pcl2', 'q6.13_SLEEP_pcl2',
                  'q6.14_ANGER_pcl2', 'q6.15_CONC_pcl2', 'q6.16_HYPER_pcl2', 'q6.17_STRTL_pcl2']

In [6]:
len(features)

92

In [7]:
discrete_features = ["age", "highschool_diploma", "dyslexia", "ADHD", "phq1", "lot1",
                    "trait1", "state1", "PCL1", "PCL_Broad1", "PCL_Strict1", "phq2", "lot2",
                    "trait2", "state2", "PCL2", "PCL_Broad2", "PCL_Strict2", "cd_risc1", "active_coping1",
                    "planning1", "positive_reframing1", "acceptance1", "humor1", "religion1",
                    "emotional_support1", "instrumental_support1", "self_distraction1", "denial1",
                    "venting1", "substance_use1", "behavioral_disengagement1", "self_blame1", "active_coping2",
                    "planning2", "positive_reframing2", "acceptance2", "humor2", "religion2", "emotional_support2",
                    "instrumental_support2", "self_distraction2", "denial2", "venting2", "substance_use2",
                    "behavioral_disengagement2", "self_blame2", "trauma_history8_1",
                    'q6.1_INTRU_pcl1', 'q6.2_DREAM_pcl1', 'q6.3_FLASH_pcl1', 'q6.4_UPSET_pcl1',
                    'q6.5_PHYS_pcl1', 'q6.6_AVTHT_pcl1', 'q6.7_AVSIT_pcl1', 'q6.8_AMNES_pcl1', 'q6.9_DISINT_pcl1',
                    'q6.10_DTACH_pcl1', 'q6.11_NUMB_pcl1', 'q6.12_FUTRE_pcl1', 'q6.13_SLEEP_pcl1',
                    'q6.14_ANGER_pcl1', 'q6.15_CONC_pcl1', 'q6.16_HYPER_pcl1', 'q6.17_STRTL_pcl1',
                    'q6.1_INTRU_pcl2', 'q6.2_DREAM_pcl2', 'q6.3_FLASH_pcl2', 'q6.4_UPSET_pcl2',
                    'q6.5_PHYS_pcl2', 'q6.6_AVTHT_pcl2', 'q6.7_AVSIT_pcl2', 'q6.8_AMNES_pcl2', 'q6.9_DISINT_pcl2',
                    'q6.10_DTACH_pcl2', 'q6.11_NUMB_pcl2', 'q6.12_FUTRE_pcl2', 'q6.13_SLEEP_pcl2',
                    'q6.14_ANGER_pcl2', 'q6.15_CONC_pcl2', 'q6.16_HYPER_pcl2', 'q6.17_STRTL_pcl2',
                    'intrusion_cutoff', 'avoidance_cutoff', 'hypertention_cutoff', 'regression_cutoff_50']

In [8]:
genome_features = ["HML_5HTT", "HL_MAOA", "HML_NPY", "COMT_Hap1_recode",
                "COMT_Hap2_recode", "COMT_Hap1_LvsMH", "HML_FKBP5"]
continuous_features = ["T1Acc1t", "T1Acc1n", "T1bias"]

In [9]:
df1 = df[features + ['PCL_Strict3', 'intrusion_cutoff', 'avoidance_cutoff', 'hypertention_cutoff', 'regression_cutoff_50']]
df1 = df1.dropna(thresh=52)

## fill the missing values

In [10]:
#df[genome_features] = df[genome_features].fillna(0)
#df[continuous_features] = df[continuous_features].fillna(df[continuous_features].mean())
#df[discrete_features] = df[discrete_features].fillna(df[discrete_features].median())
df1[genome_features] = df1[genome_features].fillna(0)
df1[continuous_features] = df1[continuous_features].fillna(df1[continuous_features].mean())
df1[discrete_features] = df1[discrete_features].fillna(df1[discrete_features].median())


## cut off the test set

In [11]:
X, X_out, Y, y_out = train_test_split(df1[features + ['intrusion_cutoff', 'avoidance_cutoff', 'hypertention_cutoff', 'regression_cutoff_50']], df1['PCL_Strict3'],
                                      test_size=0.25, random_state=271828, stratify=df1['PCL_Strict3'])


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, stratify=Y, random_state=271828)
cv = StratifiedKFold(5, random_state=271828)  

## the Model

## CV model of roc auc 

In [13]:
for loss_weight in [ 0.1, 0.75, 0.3]:
    for epochs in [200, 300]:
        for layer3 in [20]:
            for layer2 in [170]:
                layer1 = 170
                dr = 0.75
                activation = 'elu'
                batch = 1
                
                print("layer1=", layer1,"layer2=", layer2, "epochs=", epochs, "activation=", activation,
                     "batch=",batch, "dr=", dr, "loss_weight=",loss_weight, "layer3=", layer3)
                
                scores_f = []
                scores_p = []
                scores_r = []
                scores_auc = []
                
                for train, test in cv.split(X_train, y_train):
                    X_train_cv = X_train.iloc[train]
                    y_train_cv = y_train.iloc[train]

                    visible = Input(shape=(92,))
                    x = Dense(layer1, activation=activation)(visible)
                    if batch:
                        x = BatchNormalization()(x)
                    
                    x = Dropout(dr)(x)
                    if layer2 > 0:
                        x = Dense(layer2, activation=activation)(x)
                        if batch:
                            x = BatchNormalization()(x)

                        x = Dropout(dr)(x)


                    if layer3 > 0:
                        x = Dense(layer3, activation=activation)(x)
                        if batch:
                            x = BatchNormalization()(x)
                        
                        x = Dropout(dr)(x)

                    output_intrusion = Dense(1, activation='sigmoid')(x)
                    output_avoid = Dense(1, activation='sigmoid')(x)
                    output_hype = Dense(1, activation='sigmoid')(x)
                    output_50 = Dense(1, activation='sigmoid')(x)
                    output_diagnosis = Dense(1, activation='sigmoid')(x)

                    model = Model(inputs=visible, outputs=[output_intrusion, output_avoid, output_hype, output_50, output_diagnosis])

                    model.compile(loss=["binary_crossentropy"] * 5,
                              optimizer='adam', loss_weights = [loss_weight] * 4 + [1])
                    
                    t = [X_train_cv['intrusion_cutoff'], X_train_cv['avoidance_cutoff'],X_train_cv['hypertention_cutoff'],
                         X_train_cv['regression_cutoff_50'], y_train_cv]
                    
                    X_train_cv.drop(['intrusion_cutoff', 'avoidance_cutoff', 'hypertention_cutoff', 'regression_cutoff_50'], axis=1, inplace=True)
                    
                    class_weight = [{0: 1.,1: 10}]*3 + [{0: 1.,1: 25}]*2
                
                    
                    model.fit(X_train_cv, t, epochs=epochs, verbose=0, class_weight=class_weight)
                    
                    y_pred = model.predict(X_train[features].iloc[test])[4] >= 0.5

                    scores_f.append(f1_score(y_train.iloc[test], y_pred))
                    scores_p.append(precision_score(y_train.iloc[test], y_pred))
                    scores_r.append(recall_score(y_train.iloc[test], y_pred))
                    scores_auc.append(roc_auc_score(y_train.iloc[test], y_pred))
                ### results
                print("scores_f=",np.mean(scores_f))
                #print("scores_p=",scores_p)
                #print("scores_r=",scores_r)
                #print("scores_auc=",scores_auc)
                print("mean auc=",np.mean(scores_auc))

layer1= 170 layer2= 170 epochs= 200 activation= elu batch= 1 dr= 0.75 loss_weight= 0.1 layer3= 20


KeyboardInterrupt: 

In [ ]:
for dr in [0.75]:
    for epochs in [300]:
        for layer2 in [5, 30]:
            for layer1 in [90, 35, 15]:
                activation = 'tanh'
                batch = 1
                loss_weight = 0.5
                print("layer1=", layer1,"layer2=", layer2, "epochs=", epochs, "activation=", activation,
                     "batch=",batch, "dr=", dr, "loss_weight=",loss_weight)
                
                scores_f = []
                scores_p = []
                scores_r = []
                scores_auc = []
                
                for train, test in cv.split(X_train, y_train):
                    X_train_cv = X_train.iloc[train]
                    y_train_cv = y_train.iloc[train]

                    visible = Input(shape=(92,))
                    x = Dense(layer1, activation=activation)(visible)
                    if batch:
                        x = BatchNormalization()(x)
                    
                    x = Dropout(dr)(x)


                    if layer2 > 0:
                        x = Dense(layer2, activation=activation)(x)
                        if batch:
                            x = BatchNormalization()(x)
                        
                        x = Dropout(dr)(x)

                    output_intrusion = Dense(1, activation='sigmoid')(x)
                    output_avoid = Dense(1, activation='sigmoid')(x)
                    output_hype = Dense(1, activation='sigmoid')(x)
                    output_50 = Dense(1, activation='sigmoid')(x)
                    output_diagnosis = Dense(1, activation='sigmoid')(x)

                    model = Model(inputs=visible, outputs=[output_intrusion, output_avoid, output_hype, output_50, output_diagnosis])

                    model.compile(loss=["binary_crossentropy"] * 5,
                              optimizer='adam', loss_weights = [loss_weight] * 4 + [1])
                    
                    t = [X_train_cv['intrusion_cutoff'], X_train_cv['avoidance_cutoff'],X_train_cv['hypertention_cutoff'],
                         X_train_cv['regression_cutoff_50'], y_train_cv]
                    
                    X_train_cv.drop(['intrusion_cutoff', 'avoidance_cutoff', 'hypertention_cutoff', 'regression_cutoff_50'], axis=1, inplace=True)
                    
                    class_weight = [{0: 1.,1: 35}]*5
                
                    
                    model.fit(X_train_cv, t, epochs=epochs, verbose=0, class_weight=class_weight)
                    
                    y_pred = model.predict(X_train[features].iloc[test])[4] >= 0.5

                    scores_f.append(f1_score(y_train.iloc[test], y_pred))
                    scores_p.append(precision_score(y_train.iloc[test], y_pred))
                    scores_r.append(recall_score(y_train.iloc[test], y_pred))
                    scores_auc.append(roc_auc_score(y_train.iloc[test], y_pred))
                ### results
                print("scores_f=",np.mean(scores_f))
                #print("scores_p=",scores_p)
                #print("scores_r=",scores_r)
                #print("scores_auc=",scores_auc)
                print("mean auc=",np.mean(scores_auc))

layer1= 90 layer2= 5 epochs= 300 activation= tanh batch= 1 dr= 0.75 loss_weight= 0.5
scores_f= 0.14608499481738918
mean auc= 0.6037625754527163
layer1= 35 layer2= 5 epochs= 300 activation= tanh batch= 1 dr= 0.75 loss_weight= 0.5
scores_f= 0.10597016360729442
mean auc= 0.48
layer1= 15 layer2= 5 epochs= 300 activation= tanh batch= 1 dr= 0.75 loss_weight= 0.5
scores_f= 0.10882730646443728
mean auc= 0.4940845070422535
layer1= 90 layer2= 30 epochs= 300 activation= tanh batch= 1 dr= 0.75 loss_weight= 0.5
scores_f= 0.2486224712107065
mean auc= 0.7592454728370222
layer1= 35 layer2= 30 epochs= 300 activation= tanh batch= 1 dr= 0.75 loss_weight= 0.5
scores_f= 0.2268108388999163
mean auc= 0.7270020120724346
layer1= 15 layer2= 30 epochs= 300 activation= tanh batch= 1 dr= 0.75 loss_weight= 0.5


In [1]:
for dr in [0.75]:
    for epochs in [200]:
        for layer2 in [5, 30]:
            for layer1 in [90, 35, 15]:
                activation = 'elu'
                batch = 1
                loss_weight = 0.75
                print("layer1=", layer1,"layer2=", layer2, "epochs=", epochs, "activation=", activation,
                     "batch=",batch, "dr=", dr, "loss_weight=",loss_weight)
                
                scores_f = []
                scores_p = []
                scores_r = []
                scores_auc = []
                
                for train, test in cv.split(X_train, y_train):
                    X_train_cv = X_train.iloc[train]
                    y_train_cv = y_train.iloc[train]

                    visible = Input(shape=(92,))
                    x = Dense(layer1, activation=activation)(visible)
                    if batch:
                        x = BatchNormalization()(x)
                    
                    x = Dropout(dr)(x)


                    if layer2 > 0:
                        x = Dense(layer2, activation=activation)(x)
                        if batch:
                            x = BatchNormalization()(x)
                        
                        x = Dropout(dr)(x)

                    output_intrusion = Dense(1, activation='sigmoid')(x)
                    output_avoid = Dense(1, activation='sigmoid')(x)
                    output_hype = Dense(1, activation='sigmoid')(x)
                    output_50 = Dense(1, activation='sigmoid')(x)
                    output_diagnosis = Dense(1, activation='sigmoid')(x)

                    model = Model(inputs=visible, outputs=[output_intrusion, output_avoid, output_hype, output_50, output_diagnosis])

                    model.compile(loss=["binary_crossentropy"] * 5,
                              optimizer='adam', loss_weights = [loss_weight] * 4 + [1])
                    
                    t = [X_train_cv['intrusion_cutoff'], X_train_cv['avoidance_cutoff'],X_train_cv['hypertention_cutoff'],
                         X_train_cv['regression_cutoff_50'], y_train_cv]
                    
                    X_train_cv.drop(['intrusion_cutoff', 'avoidance_cutoff', 'hypertention_cutoff', 'regression_cutoff_50'], axis=1, inplace=True)
                    
                    class_weight = [{0: 1.,1: 15}]*5
                
                    
                    model.fit(X_train_cv, t, epochs=epochs, verbose=0, class_weight=class_weight)
                    
                    y_pred = model.predict(X_train[features].iloc[test])[4] >= 0.5

                    scores_f.append(f1_score(y_train.iloc[test], y_pred))
                    scores_p.append(precision_score(y_train.iloc[test], y_pred))
                    scores_r.append(recall_score(y_train.iloc[test], y_pred))
                    scores_auc.append(roc_auc_score(y_train.iloc[test], y_pred))
                ### results
                print("scores_f=",np.mean(scores_f))
                #print("scores_p=",scores_p)
                #print("scores_r=",scores_r)
                #print("scores_auc=",scores_auc)
                print("mean auc=",np.mean(scores_auc))

layer1= 90 layer2= 5 epochs= 200 activation= elu batch= 1 dr= 0.75 loss_weight= 0.75


NameError: name 'cv' is not defined

In [ ]:
from sklearn.metrics import roc_curve, auc


#X_test.drop(['intrusion_cutoff', 'avoidance_cutoff', 'hypertention_cutoff'], axis=1, inplace=True)
probs = model.predict(X_test)[3]
preds = probs
fpr, tpr, threshold = roc_curve(y_test, preds)
roc_auc = auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
plt.hist(preds[y_test==0], bins=50, color="b")
plt.hist(preds[y_test==1], bins=20, alpha=0.8, color="r")